## k-Means Clustering

example implementation of algorithm for interactive use

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def savedata(X, y, fname='nums.csv'):
    n = y.shape[0]
    y=y.reshape(n,1)
    res = np.append(X, y, axis=1)
    np.savetxt(fname,res,delimiter=",", header="x,y,label", comments="", fmt='%.5f')
def readdata(fname='nums.csv'):
    res = np.loadtxt(fname,skiprows=1,delimiter=',')
    X = res[:,:2]
    y = res[:,2]
    return X,y

In [ ]:
X,y = readdata('nums_blob_3.csv')
X.shape

In [ ]:
# initialize centers
# take avg and randomly +- std-dev
nc = 3
xavg,xstd = X.mean(axis=0),X.std(axis=0)
centers=(np.random.random((nc,2))*2-1)*xstd+xavg
print(centers)

In [ ]:
# helper functions
def findclosest( p, pl ):
    """ 
    find closest point in list
    args: p = 2D point, pl = list of 2D points
    return: index of closest point in pl
    """
    d = (p[0]-pl[:,0])**2 + (p[1]-pl[:,1])**2
    i = np.argmin(d)
    return i

def mean_plist( pl, centroids ):
    """ 
    determine first centroid-index list of points closest to respective centroid
    and calculate then new mean centroid for for each centroid-index list of points
    args: pl = list of 2D points, centroids = list of centroid 2D points
    return: list of new centroids, index list with centroid association, loss per centroid
    """
    ilist = np.array([findclosest(v,centroids) for v in pl])
    centnew = np.array([pl[ilist==i].mean(axis=0) for i in range(len(centroids))])
    loss = np.array([((pl[ilist==i]-centnew[i])**2).sum() for i in range(len(centroids))])
    return centnew,ilist,loss

In [ ]:
# plot initial positioning
plt.scatter(X[:, 0], X[:, 1],s=10)#, c=y,cmap='rainbow');
plt.scatter(*centers.T,s=30,color='k')

In [ ]:
cnew = centers

In [ ]:
# repeatedly execute until convergence
cnew,ilist,loss = mean_plist( X, cnew )
cnew,loss,loss.sum()

In [ ]:
# plot result
plt.scatter(X[:, 0], X[:, 1],s=10, c=ilist,cmap='rainbow');
plt.scatter(*cnew.T,s=30,color='k')

---

## k-Means from sklearn

In [ ]:
from sklearn.cluster import KMeans


In [ ]:
KMeans?

In [ ]:

kmeans = KMeans(n_clusters=3, n_init = 10, random_state=30) #predicts 0,1,2
kmeans.fit(X)
yb_kmeans = kmeans.predict(X)
centersb = kmeans.cluster_centers_ # the resulting centers
inertia = kmeans.inertia_  # the loss

In [ ]:
centersb

In [ ]:
kmeans.inertia_

#### Elbow plot

In [ ]:
inertias = []
for k in range(2, 8):
    kmeans = KMeans(n_clusters=k, n_init = 10)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)

In [ ]:
fig = plt.figure(figsize=(8, 6))
plt.plot(range(2, 8), inertias)
plt.title('Elbow curve');
plt.xlabel('Number of clusters $k$', fontsize = 14);
plt.ylabel('$k$-means cost function', fontsize = 14);


### More complex case with overlapping blobs

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans


In [ ]:
# create 3 blobs
X1b, y1b = make_blobs(n_samples=200, centers=[(1.25,1)],
                       cluster_std=0.2, random_state=1)

X2b, y2b = make_blobs(n_samples=400, centers=[(0,1)],
                       cluster_std=0.5, random_state=2)

X3b, y3b = make_blobs(n_samples=200, centers=[(-1.25,1)],
                       cluster_std=0.2, random_state=3)

In [ ]:
# combine blobs
Xb = np.vstack([X1b,X2b,X3b])
yb = np.concatenate([y1b,y2b+1,y3b+2])
Xb.shape, yb.shape

In [ ]:
# plot result
plt.scatter(Xb[:, 0], Xb[:, 1],s=10, c=yb,cmap='rainbow');

In [ ]:
from mltools import visualize_classifier
nc = 3
model = KMeans(n_clusters=nc, n_init = 10, random_state=30) #predicts 0,1,2
model.fit(Xb)
# trick to get color assiciation right
clist = [ model.predict(Xb[yb==i]).mean().round() for i in range(nc)]
yc = np.array([ clist[i] for i in yb ])

# now visualize points and mode prediction
visualize_classifier(model, Xb, yc, cmap='rainbow')

#### and the elbow plot ...

In [ ]:
nmax = 10
inertiasb = []
for k in range(2, nmax):
    kmeans = KMeans(n_clusters=k, n_init = 10)
    kmeans.fit(Xb)
    inertiasb.append(kmeans.inertia_)

plt.plot(range(2, nmax), inertiasb)
#plt.grid(True)
plt.title('Elbow curve for blobs');
plt.xlabel('Number of clusters $k$', fontsize = 14);
plt.ylabel('$k$-means cost function', fontsize = 14);


## Gaussian mixture model

In [ ]:
from sklearn import mixture

In [ ]:
from mltools import visualize_classifier
nc = 3
model = mixture.GaussianMixture(n_components=3, covariance_type='full',random_state=41)
model.fit(Xb)

# trick to get color association right
clist = [ model.predict(Xb[yb==i]).mean().round() for i in range(nc)]
yc = np.array([ clist[i] for i in yb ])


# now visualize points and mode prediction
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.set_xlim(-2.5, 2.5)
ax.set_ylim(-0.5,2.5)
visualize_classifier(model, Xb, yc, cmap='rainbow', ax=ax )


#### GMM model selection

The selection of the appropriate model is a bit more complicated for GMM
- there are various options what type of Gaussians are used
- and obviously the number of components

A tradeoff must be made between complexity of model and the loss. 
Information theory provides the Bayes Information Criterion (BIC) as a quantitative
estimator, see [**SKlearn documentation**](https://scikit-learn.org/stable/auto_examples/mixture/plot_gmm_selection.html#data-generation) for a detailed example and discussion. 

<div class="alert alert-block alert-success">
    <h2>Exercise 1</h2>
    Read the data from 'nums_blob_X.csv' .
    Use k-Means clustering and elbow plot to determine number of clusters
</div>    


In [ ]:
X,y = readdata('nums_blob_X.csv')
X.shape